In [ ]:
import os
from typing import Iterable, Set

# =========================================================
# =================== CONFIG GLOBAL =======================
# =========================================================

BASE_DIR = os.path.abspath(".")

TARGETS = [
    {
        "nombre": "backend",
        "ruta": os.path.join(BASE_DIR, "backend"),
        "salida": "context_backend_APPRAG.txt",
        "extensiones": {".py", ".txt", ".yaml", "Dockerfile", ".env.example",},
        "archivos_excluidos": set(),
    },
    {
        "nombre": "frontend",
        "ruta": os.path.join(BASE_DIR, "frontend"),
        "salida": "context_frontend_APPRAG.txt",
        "extensiones": {
            ".tsx",
            ".css",
            ".ts",
            ".js",
            ".conf",
            "Dockerfile",
            ".env.example",
        },
        "archivos_excluidos": set(),
    },
]

# Carpetas excluidas globales
CARPETAS_EXCLUIDAS: Set[str] = {
    ".github",
    ".vscode",
    "env",
    "node_modules",
    "agents",
    "agent",
}

# =========================================================
# ===================== CORE LOGIC ========================
# =========================================================

def extension_valida(nombre_archivo: str, extensiones: Iterable[str]) -> bool:
    """
    Comprueba si el archivo coincide con las extensiones permitidas.
    Soporta tanto '.py' como 'Dockerfile'.
    """
    _, ext = os.path.splitext(nombre_archivo)

    if ext.lower() in extensiones:
        return True

    if nombre_archivo in extensiones:
        return True

    return False


def procesar_target(config: dict) -> None:
    ruta_base = config["ruta"]
    archivo_salida = config["salida"]
    extensiones = config["extensiones"]
    archivos_excluidos = config["archivos_excluidos"]

    if not os.path.exists(ruta_base):
        print(f"⚠️ Ruta no encontrada: {ruta_base}")
        return

    print(f"🚀 Procesando: {config['nombre']}")

    with open(archivo_salida, "w", encoding="utf-8") as out:
        for root, dirs, files in os.walk(ruta_base):
            # excluir carpetas IN-PLACE (importante para rendimiento)
            dirs[:] = [d for d in dirs if d not in CARPETAS_EXCLUIDAS]

            for file in files:
                if file in archivos_excluidos:
                    continue

                if not extension_valida(file, extensiones):
                    continue

                ruta_completa = os.path.join(root, file)
                ruta_completa = os.path.relpath(ruta_completa, BASE_DIR)

                out.write(f"### ARCHIVO: {ruta_completa}\n")

                try:
                    with open(ruta_completa, "r", encoding="utf-8", errors="ignore") as f:
                        out.write(f.read())
                        out.write("\n---\n\n")
                except Exception as e:
                    out.write(f"[ERROR AL LEER ARCHIVO: {e}]\n\n")

    print(f"✅ Generado: {archivo_salida}")


# =========================================================
# ======================= MAIN ============================
# =========================================================

def main():
    for target in TARGETS:
        procesar_target(target)

    print("\n🎉 Proceso completo.")


main()

🚀 Procesando: backend
✅ Generado: context_backend_APPRAG.txt
🚀 Procesando: frontend
✅ Generado: context_frontend_APPRAG.txt

🎉 Proceso completo.


In [ ]:
# TARGETS.append({
#     "nombre": "nuevo_modulo",
#     "ruta": os.path.join(BASE_DIR, "nuevo"),
#     "salida": "context_nuevo.txt",
#     "extensiones": {".py"},
#     "archivos_excluidos": set(),
# })